# `metadsl`: seperating API from execution

In [6]:
from __future__ import annotations

import metadsl
import metadsl_core
import metadsl_visualize

(metadsl_core.arange(10000) + metadsl_core.arange(10000))[5].to_ndarray()

Typez(definitions=None, nodes={'-7583180538730774302': ['238', PrimitiveNode(type='int', repr='10000')], '845730279043314766': ['7', CallNode(function='arange', type_params=None, args=['-7583180538730774302'], kwargs=None)], '9100461696994788422': ['8', CallNode(function='NDArrayCompat.__add__', type_params=None, args=['845730279043314766', '845730279043314766'], kwargs=None)], '4300010369523878515': ['229', PrimitiveNode(type='int', repr='5')], '7475593968739951540': ['4', CallNode(function='NDArrayCompat.__getitem__', type_params=None, args=['9100461696994788422', '4300010369523878515'], kwargs=None)], '-7219239887681001304': ['5', CallNode(function='NDArrayCompat.to_ndarray', type_params=None, args=['7475593968739951540'], kwargs=None)], '-8522665909237615706': ['9', CallNode(function='Converter.convert', type_params={'T': DeclaredTypeInstance(type='NDArray', params=None)}, args=['9100461696994788422'], kwargs=None)], '-5876881260713993123': ['11', CallNode(function='Converter.conve

array(10)

## Python

In [7]:
def arange_add_and_index():
    x = [i for i in range(10000)]
    y = [i + i for i in x]
    return y[5]

%time arange_add_and_index()

CPU times: user 1.17 ms, sys: 1 µs, total: 1.17 ms
Wall time: 1.18 ms


10

![](https://user-images.githubusercontent.com/1186124/68094720-43935780-fe71-11e9-9169-533478dd7e02.png)

### NumPy

The classic approach of speeding up Python for scientific workloads, shown in tools like NumPy and Pandas, keeps this architecture, but it just optimizes chunks of it. You write specialized optimized instructions for common workloads and call them when we need them. (LaPack, Fortran, C, Cython).

In [8]:
import numpy

def add_and_index_np():
    x = numpy.arange(10000)
    y = (x + x)
    return y[5]

%time add_and_index_np()

CPU times: user 84 µs, sys: 79 µs, total: 163 µs
Wall time: 154 µs


10

![](https://user-images.githubusercontent.com/1186124/68094718-42fac100-fe71-11e9-83d4-23d3bb5a47b8.png)

## Single Core = Von Neuman Architecture

[![](https://user-images.githubusercontent.com/1186124/68030190-43d3fd00-fc8f-11e9-8cfd-296d04529f42.png)](https://github.com/Quansight-Labs/metadsl/issues/77#issuecomment-548800090)

> A **von Neumann language is any of those programming languages that are high-level abstract isomorphic copies of von Neumann architectures**. As of 2009, **most current programming languages fit into this description**, likely as a consequence of the extensive domination of the von Neumann computer architecture during the past 50 years. 

Two ways to increase performance:

* **Hardware**: Run instructions faster
* **Software**: Run less instructions

## Hardware Slowdown

[![](https://www.karlrupp.net/wp-content/uploads/2018/02/42-years-processor-trend.png)](https://web.archive.org/web/20191013185855/https://www.karlrupp.net/2018/02/42-years-of-microprocessor-trend-data/)

* [**Moore's law**](https://en.wikipedia.org/wiki/Moore%27s_law): # transistors doubles every two years
* **Denard scaling**: As transistors get smaller, power stays proportionate to area

> The exponential processor transistor growth predicted by Moore does not always translate into exponentially greater practical CPU performance. Since around 2005–2007, **Dennard scaling appears to have broken down**, so even though Moore's law continued for several years after that, it has not yielded dividends in improved performance. The primary reason cited for the breakdown is that at small sizes, current leakage poses greater challenges, and also causes the chip to heat up, which creates a threat of thermal runaway and therefore, further increases energy costs.
> 
> The breakdown of Dennard scaling **prompted a switch among some chip manufacturers to a greater focus on multicore processors**, but the gains offered by switching to more cores are lower than the gains that would be achieved had Dennard scaling continued. In another departure from Dennard scaling, Intel microprocessors adopted a non-planar tri-gate FinFET at 22 nm in 2012 that is faster and consumes less power than a conventional planar transistor.

Moore's Law + Denard Scaling = [Koomey's Law](https://en.wikipedia.org/wiki/Koomey%27s_law): computations per joule double every 1.57 years

> By the second law of thermodynamics and Landauer's principle, irreversible computing cannot continue to be made more energy efficient forever. As of 2011, computers have a computing efficiency of about 0.00001%.[13] **Assuming that the energy efficiency of computing will continue to double every 1.57 years, the Landauer bound will be reached in 2048**. Thus, after about 2048, Koomey's law can no longer hold. 

#### Moving to non Von Neuman Hardware

* CPU pipelining
* Caching
* Multiprocessing
* GPUs
* ASICs
* FPGAs

### Software Slowdown

![](https://d3i71xaburhd42.cloudfront.net/38c1f34efedab0e04945ab1abb6e0087abdd4894/1-Figure1-1.png)

**John Backus** in his **1977 ACM Turing Award lecture**:

> Surely there must be a less primitive way of making big changes in the store than by pushing vast numbers of words back and forth through the **von Neumann bottleneck**. Not only is this tube a literal bottleneck for the data traffic of a problem, but, more importantly, it is
>
> # an intellectual bottleneck
> ## that has kept us tied to word-at-a-time thinking
> ### instead of encouraging us to think in terms of the larger conceptual units of the task at hand
>
> Thus programming is basically planning and detailing the enormous traffic of words through the von Neumann bottleneck, and much of that traffic concerns not significant data itself, but where to find it.


#### Leads to non Von Neuman Software

[![](https://www.easy-tensorflow.com/files/1_1.gif)](https://www.easy-tensorflow.com/tf-tutorials/basics/graph-and-session)

## This is hard!

![](https://user-images.githubusercontent.com/1186124/68095556-768d1980-fe78-11e9-872f-747c0d1c80f4.png)

### Community Open Source

![](https://user-images.githubusercontent.com/1186124/68102006-b9fa7e80-fe9e-11e9-93d0-be7c4c2c2309.png)

**decentralized cooperation, accessability, and performance**

We have tools in Python, just need to connect them without vendor lock in:

* Deep learning frameworks, like Tensorflow
* Whole program optimizations, like Numba
* Distributed computing, like Dask
* Functional programming

Need tool to write domain specific languages in Python.

### `metadsl`

In [7]:
class Array(metadsl.Expression):
    @metadsl.expression
    def __getitem__(self, idx: int) -> Array:
        ...
    
    @metadsl.expression
    def __add__(self, other: Array) -> Array:
        ...

@metadsl.expression
def arange(n: int) -> Array:
    ...

In [8]:
def double_and_index(a):
    return (a + a)[5]

In [18]:
m.Expression??

Init signature:
m.Expression(
    function: 'typing.Callable',
    args: 'typing.List[object]',
    kwargs: 'typing.Dict[str, object]',
) -> None
Source:        
class Expression(GenericCheck):
    """
    Top level object.
    Subclass this type and provide relevent methods for your type. Do not add any fields.

    Properties:

    Calling the function, after replacing the typevars in it (if it is a bound method),
    with the args and kwargs should resualt in an equivalent expression:

    replace_fn_typevars(self.function, self.typevars)(*self.args, **self.kwargs) == self
    
    The return type of the function, inferred by replacing the typevars in and with these args and kwargs,
    should match the type of the expression. If the return type of the function is not subclass of expression,
    then this should be a PlaceholderExpression of that type.
    """

    function: typing.Callable
    args: typing.List[object]
    kwargs: typing.Dict[str, object]

    def __str__(self):
  

In [15]:
res = double_and_index(arange(10))
res.function

Array.__getitem__

In [12]:
import numpy

@metadsl.expression
def array_from_np(a: numpy.ndarray) -> Array:
    ...


numpy_rules = metadsl.RulesRepeatFold()
metadsl.execute.default_rule = numpy_rules


@numpy_rules.append
@metadsl.rule
def np_arange_rule(i: int) -> metadsl.R[Array]:
    return arange(i), lambda: array_from_np(numpy.arange(i))


@numpy_rules.append
@metadsl.rule
def np_add_rule(l: numpy.ndarray, r: numpy.ndarray) -> metadsl.R[Array]:
    return array_from_np(l) + array_from_np(r), lambda: array_from_np(l + r)

@numpy_rules.append
@metadsl.rule
def np_getitem_rule(a: numpy.ndarray, i: int) -> metadsl.R[Array]:
    return array_from_np(a)[i], lambda: array_from_np(numpy.asarray(a[i]))

double_and_index(arange(10))

Typez(definitions=None, nodes={'-2343543019907976475': ['0', PrimitiveNode(type='int', repr='10')], '-8021972163509979648': ['1', CallNode(function='arange', type_params=None, args=['-2343543019907976475'], kwargs=None)], '2760859927190853373': ['2', CallNode(function='Array.__add__', type_params=None, args=['-8021972163509979648', '-8021972163509979648'], kwargs=None)], '-2343543019896068700': ['3', PrimitiveNode(type='int', repr='5')], '1140167751888136435': ['4', CallNode(function='Array.__getitem__', type_params=None, args=['2760859927190853373', '-2343543019896068700'], kwargs=None)], '2952535132544636725': ['5', PrimitiveNode(type='ndarray', repr='[0 1 2 3 4 5 6 7 8 9]')], '4435428548954370711': ['1', CallNode(function='array_from_np', type_params=None, args=['2952535132544636725'], kwargs=None)], '-5097058542522012973': ['2', CallNode(function='Array.__add__', type_params=None, args=['4435428548954370711', '4435428548954370711'], kwargs=None)], '1485754249729899440': ['4', CallN

#### Symbolic

In [13]:
@metadsl.expression
def symbolic_array(a: str) -> Array:
    ...

symbolic_rules = metadsl.RulesRepeatFold()
metadsl.execute.default_rule = symbolic_rules

    
@symbolic_rules.append
@metadsl.rule
def symbolic_add_rule(l: str, r: str) -> metadsl.R[Array]:
    return symbolic_array(l) + symbolic_array(r), lambda: symbolic_array(f"({l} + {r})")

@symbolic_rules.append
@metadsl.rule
def symbolic_getitem_rule(a: str, i: int) -> metadsl.R[Array]:
    return symbolic_array(a)[i], lambda: symbolic_array(f"{a}[{i}]")

double_and_index(symbolic_array("a"))

Typez(definitions=None, nodes={'-8898239426087068317': ['0', PrimitiveNode(type='str', repr='a')], '-7453694020426302651': ['1', CallNode(function='symbolic_array', type_params=None, args=['-8898239426087068317'], kwargs=None)], '648306428635280927': ['2', CallNode(function='Array.__add__', type_params=None, args=['-7453694020426302651', '-7453694020426302651'], kwargs=None)], '-2343543019896068700': ['3', PrimitiveNode(type='int', repr='5')], '5240157556638217626': ['4', CallNode(function='Array.__getitem__', type_params=None, args=['648306428635280927', '-2343543019896068700'], kwargs=None)], '7266168266899572383': ['5', PrimitiveNode(type='str', repr='(a + a)')], '4450278409172312431': ['2', CallNode(function='symbolic_array', type_params=None, args=['7266168266899572383'], kwargs=None)], '380119097328660081': ['4', CallNode(function='Array.__getitem__', type_params=None, args=['4450278409172312431', '-2343543019896068700'], kwargs=None)], '-4487234170864159322': ['6', PrimitiveNode

### Optimization

In [14]:
@metadsl.rule
def optimize_getitem_add(l: Array, r: Array, idx: int) -> metadsl.R[Array]:
    return (
        (l + r)[idx],
        l[idx] + r[idx]
    )

metadsl.execute.default_rule = metadsl.RuleInOrder(
    optimize_getitem_add,
    symbolic_rules
)
double_and_index(symbolic_array("a"))

Typez(definitions=None, nodes={'-8898239426087068317': ['5', PrimitiveNode(type='str', repr='a')], '-7453694020426302651': ['6', CallNode(function='symbolic_array', type_params=None, args=['-8898239426087068317'], kwargs=None)], '648306428635280927': ['2', CallNode(function='Array.__add__', type_params=None, args=['-7453694020426302651', '-7453694020426302651'], kwargs=None)], '-2343543019896068700': ['7', PrimitiveNode(type='int', repr='5')], '5240157556638217626': ['4', CallNode(function='Array.__getitem__', type_params=None, args=['648306428635280927', '-2343543019896068700'], kwargs=None)], '4726567948630348247': ['8', CallNode(function='Array.__getitem__', type_params=None, args=['-7453694020426302651', '-2343543019896068700'], kwargs=None)], '-5454009444032897499': ['4', CallNode(function='Array.__add__', type_params=None, args=['4726567948630348247', '4726567948630348247'], kwargs=None)], '6273367703462737161': ['9', PrimitiveNode(type='str', repr='a[5]')], '4103583956861880946'

#### Abstract Arrays

In [15]:
VecInt = metadsl_core.Vec[metadsl_core.Integer]
IdxFn = metadsl_core.Abstraction[metadsl_core.Vec[metadsl_core.Integer], metadsl_core.Integer]

class AbstractArray(metadsl.Expression):
    @metadsl.expression
    def create(shape: VecInt, idx_fn: IdxFn) -> AbstractArray:
        ...

    @metadsl.expression
    @property
    def shape(self) -> VecInt:
        ...
        
    @metadsl.expression
    def __getitem__(self, indx: VecInt) -> metadsl_core.Integer:
        ...

    @metadsl.expression
    def __add__(self, other: AbstractArray) -> AbstractArray:
        @metadsl_core.Abstraction.from_fn
        def new_idx_fn(idx: VecInt) -> metadsl_core.Integer:
            return self[idx] + other[idx]
    
        return AbstractArray.create(self.shape, new_idx_fn)

In [16]:
abstract_rules = metadsl.RulesRepeatFold()
@abstract_rules.append
@metadsl.rule
def abstract_getitem_rule(idx_fn: IdxFn, shape: VecInt, indx: VecInt) -> metadsl.R[metadsl_core.Integer]:
    return (
        AbstractArray.create(shape, idx_fn)[indx],
        idx_fn(indx)
    )

@abstract_rules.append
@metadsl.rule
def abstract_shape_rule(idx_fn: IdxFn, shape: VecInt) -> metadsl.R[VecInt]:
    return (
        AbstractArray.create(shape, idx_fn).shape,
        shape
    )

abstract_rules.append(metadsl.default_rule(AbstractArray.__add__))

DefaultRule(fn=AbstractArray.__add__)

In [17]:
@metadsl.expression
def array_from_abstract(abs_array: AbstractArray) -> Array:
    ...
    
@metadsl.expression
def array_from_scalar(s: metadsl_core.Integer) -> Array:
    ...
    
    
@metadsl.expression
def create_abstract_array_str(label: str) -> AbstractArray:
    ...

array_to_abstract_rules = metadsl.RulesRepeatFold()

metadsl.execute.default_rule = metadsl.RulesRepeatSequence(
    metadsl_core.all_rules,
    array_to_abstract_rules,
    abstract_rules
)

@array_to_abstract_rules.append
@metadsl.rule
def symbolic_to_abstract_rule(l: str) -> metadsl.R[Array]:
    return (
        symbolic_array(l),
        array_from_abstract(create_abstract_array_str(l))
    )

@array_to_abstract_rules.append
@metadsl.rule
def array_getitem_rule(a: AbstractArray, i: int) -> metadsl.R[Array]:
    return (
        array_from_abstract(a)[i],
        array_from_scalar(a[metadsl_core.Vec.create(metadsl_core.Integer.from_int(i))])   
    )

@array_to_abstract_rules.append
@metadsl.rule
def array_add_rule(l: AbstractArray, r: AbstractArray) -> metadsl.R[Array]:
    return (
        array_from_abstract(l) + array_from_abstract(r),
        array_from_abstract(l + r)
    )

In [18]:
double_and_index(symbolic_array("a"))

Typez(definitions=None, nodes={'-8898239426087068317': ['33', PrimitiveNode(type='str', repr='a')], '-7453694020426302651': ['1', CallNode(function='symbolic_array', type_params=None, args=['-8898239426087068317'], kwargs=None)], '648306428635280927': ['2', CallNode(function='Array.__add__', type_params=None, args=['-7453694020426302651', '-7453694020426302651'], kwargs=None)], '-2343543019896068700': ['35', PrimitiveNode(type='int', repr='5')], '5240157556638217626': ['4', CallNode(function='Array.__getitem__', type_params=None, args=['648306428635280927', '-2343543019896068700'], kwargs=None)], '6178083059846014151': ['34', CallNode(function='create_abstract_array_str', type_params=None, args=['-8898239426087068317'], kwargs=None)], '5514147066383566795': ['1', CallNode(function='array_from_abstract', type_params=None, args=['6178083059846014151'], kwargs=None)], '2214342695731065885': ['2', CallNode(function='Array.__add__', type_params=None, args=['5514147066383566795', '5514147066

### [Computing for Everybody](https://www.python.org/doc/essays/cp4e/)

![](https://aem.dropbox.com/cms/content/dam/dropbox/blog/company/2019/guido_featured.png)
> "What will happen if users can program their own computer?" We're looking forward to a future where every computer user will be able to **"open the hood" of their computer and make improvements to the applications inside**. We believe that this will eventually change the nature of software and software development tools fundamentally. 

We need to offer users friendly onramps into the ecosystem that will continue supporting new hardware and optimizations, or we will push them to vendor lock in closed ecosystems. So we need to figure out what kind of technology can support a decentralized and multi stakeholder approrach. Even inside of Google they are moving to a similar approach with MLIR. I love using Python, because it is accessable and friendly. We can shape this ecosystem to our will to make it easy for new folks to contribute and work together. 

### Errata

#### "Folding Domain-Specific Languages: Deep and Shallow Embeddings" - Jeremy Gibbons


We call this a domain specific language.

> There are two main approaches to DSLs. **Standalone DSLs** provide their own custom syntax and semantics, and standard compilation techniques are used to translate or interpret programs written in the DSL for execution. Standalone DSLs can be designed for maximal convenience to their intended users. But **the exercise can be a significant undertaking for the implementer**, involving an entirely separate ecosystem—compiler, editor, debugger, and so on—and typically also much reinven- tion of standard language features such as variables, definitions, and conditionals.

> The alternative approach is to **embed the DSL within a host GPL**, essentially as a collection of definitions written in the host language. **All the existing facilities and infrastructure of the host environment can be appropriated for the DSL**, and familiarity with the syntactic conventions and tools for the host language can be carried over to the DSL. Whereas the standalone approach is the most common one within object-oriented circles [5], the embedded approach is typically favoured by functional programmers [11]. It seems that core FP features such as algebraic datatypes and higher-order functions are extremely helpful in defining embedded DSLs; conversely, it has been said that language-oriented tasks such as DSLs are the killer application for FP.

> Amongst embedded DSLs, there are two further refinements. With **a deep embedding, terms in the DSL are implemented simply to construct an abstract syntax tree (AST)**; this tree is subsequently transformed for optimization and traversed for evaluation. With **a shallow embedding, terms in the DSL are implemented directly as the values to which they evaluate, bypassing the intermediate AST and its traversal**.

NumPy is a shallowly embedded domain specific language, since the "terms" (functions) in it are executed immidiatly. No AST is created.

`metadsl` lets you create APIs like NumPy that are deeply embedded, so that first we construct an AST. That's what we saw in the images above. Then we can transform that tree to either evaluate it direcly or compile it to another form.

So maybe it really should be `metadedsl` (meta deeply embedded domain specific language)?

Metadsl allows creating a deeply embedding languages in Python to optimize and compile them.

#### Introspection

We can start to think about semantic open research and connection with opening. Reproducible science. Makes compute data so we can understand our scientific processes better.

Benefits scientists

## Current State

#### LLVM

In [1]:
import metadsl as m
import metadsl_core as mc
import metadsl_llvm as ml
import metadsl_visualize

In [2]:
def create_metadsl_fn():

    ##
    # Constants
    ##

    int_type = ml.Type.create_int(32)
    zero = ml.Value.constant(int_type, 0)
    one = ml.Value.constant(int_type, 1)

    ##
    # Module Reference
    ##
    mod = ml.ModuleReference.create("fib")

    ##
    # Module Builder
    ##
    mod_builer = ml.ModuleBuilder.create(mod)

    ##
    # Function References
    ##
    mod_builder, fib_more_fn = ml.FunctionReference.create(
        mod_builer,
        ml.FunctionType.create(int_type, int_type, int_type, int_type),
        "fib_more",
        "fastcc",
    ).spread

    mod_builder, fib_fn = ml.FunctionReference.create(
        mod_builer, ml.FunctionType.create(int_type, int_type), "fib", "fastcc"
    ).spread

    ##
    # Function Builders
    ##
    fib_more_builder = ml.FunctionBuilder.create(fib_more_fn)
    fib_fn_builder = ml.FunctionBuilder.create(fib_fn)

    ##
    # Arguments
    ##

    fib_n = fib_fn_builder.arguments[mc.Integer.from_int(0)]
    fib_more_n = fib_more_builder.arguments[mc.Integer.from_int(0)]
    fib_more_a = fib_more_builder.arguments[mc.Integer.from_int(1)]
    fib_more_b = fib_more_builder.arguments[mc.Integer.from_int(2)]

    ##
    # Block References
    ##
    fib_fn_builder, fib_entry = ml.BlockReference.create("entry", fib_fn_builder).spread
    fib_more_builder, fib_more_entry = ml.BlockReference.create(
        "entry", fib_more_builder
    ).spread
    fib_more_builder, fib_pred_cont = ml.BlockReference.create(
        "pred_cont", fib_more_builder
    ).spread
    fib_more_builder, fib_not_pred_cont = ml.BlockReference.create(
        "not_pred_cont", fib_more_builder
    ).spread
    fib_more_builder, fib_n_eq_one = ml.BlockReference.create(
        "n_eq_one", fib_more_builder
    ).spread
    fib_more_builder, fib_n_neq_one = ml.BlockReference.create(
        "n_neq_one", fib_more_builder
    ).spread

    ##
    # Block Builders
    ##
    fib_entry_builder = ml.BlockBuilder.create(fib_entry)
    fib_more_entry_builder = ml.BlockBuilder.create(fib_more_entry)
    fib_pred_cont_builder = ml.BlockBuilder.create(fib_pred_cont)
    fib_not_pred_cont_builder = ml.BlockBuilder.create(fib_not_pred_cont)
    fib_n_eq_one_builder = ml.BlockBuilder.create(fib_n_eq_one)
    fib_n_neq_one_builder = ml.BlockBuilder.create(fib_n_neq_one)

    fib_entry_builder, fib_entry_res = fib_entry_builder.call(
        fib_more_fn, mc.Vec.create(fib_n, zero, one)
    ).spread
    fib_entry_builder = fib_entry_builder.ret(fib_entry_res)

    fib_more_entry_builder, pred_cont = fib_more_entry_builder.icmp_signed(
        ">", fib_more_n, one
    ).spread

    fib_more_entry_builder = fib_more_entry_builder.cbranch(
        pred_cont, fib_pred_cont, fib_not_pred_cont
    )

    fib_pred_cont_builder, minus1 = fib_pred_cont_builder.sub(fib_more_n, one).spread
    fib_pred_cont_builder, ab = fib_pred_cont_builder.add(fib_more_a, fib_more_b).spread

    fib_pred_cont_builder, added = fib_pred_cont_builder.call(
        fib_more_fn, mc.Vec.create(minus1, fib_more_b, ab)
    ).spread
    fib_pred_cont_builder = fib_pred_cont_builder.ret(added)

    fib_not_pred_cont_builder, n_eq_1 = fib_not_pred_cont_builder.icmp_signed(
        "==", fib_more_n, one
    ).spread
    fib_not_pred_cont_builder = fib_not_pred_cont_builder.cbranch(
        n_eq_1, fib_n_eq_one, fib_n_neq_one
    )

    fib_n_eq_one_builder = fib_n_eq_one_builder.ret(fib_more_b)

    fib_n_neq_one_builder = fib_n_neq_one_builder.ret(fib_more_a)

    ##
    # Blocks
    ##
    fib_entry_block = ml.Block.create(fib_entry, fib_entry_builder)
    fib_more_entry_block = ml.Block.create(fib_more_entry, fib_more_entry_builder)
    fib_pred_cont_block = ml.Block.create(fib_pred_cont, fib_pred_cont_builder)
    fib_not_pred_cont_block = ml.Block.create(
        fib_not_pred_cont, fib_not_pred_cont_builder
    )
    fib_n_eq_one_block = ml.Block.create(fib_n_eq_one, fib_n_eq_one_builder)
    fib_n_neq_one_block = ml.Block.create(fib_n_neq_one, fib_n_neq_one_builder)

    ##
    # Functions
    ##
    fib_fn_real = ml.Function.create(fib_fn, mc.Vec.create(fib_entry_block))
    fib_more_fn_real = ml.Function.create(
        fib_more_fn,
        mc.Vec.create(
            fib_more_entry_block,
            fib_pred_cont_block,
            fib_not_pred_cont_block,
            fib_n_eq_one_block,
            fib_n_neq_one_block,
        ),
    )

    ##
    # Module
    ##

    module_real = ml.Module.create(mod, mc.Vec.create(fib_fn_real, fib_more_fn_real))

    ##
    # CType
    ##
    c_int = ml.CType.c_int()
    c_func_type = ml.CFunctionType.create(c_int, c_int)

    return ml.compile_function(module_real, mod_builder, fib_fn, c_func_type)


In [3]:
metadsl_fn = m.execute(create_metadsl_fn())

Typez(definitions=None, nodes={'-4578469861668566468': ['272', PrimitiveNode(type='str', repr='fib')], '-8942513152654453670': ['198', CallNode(function='ModuleReference.create', type_params=None, args=['-4578469861668566468'], kwargs=None)], '-4280351851181442952': ['104', CallNode(function='ModuleBuilder.create', type_params=None, args=['-8942513152654453670'], kwargs=None)], '-8685184296869118711': ['105', PrimitiveNode(type='int', repr='32')], '-6794287655980958172': ['106', CallNode(function='Type.create_int', type_params=None, args=['-8685184296869118711'], kwargs=None)], '-1550960115321255115': ['107', CallNode(function='FunctionType.create', type_params=None, args=['-6794287655980958172', '-6794287655980958172'], kwargs=None)], '5907635427235633412': ['108', PrimitiveNode(type='str', repr='fastcc')], '-3197797570619298217': ['109', CallNode(function='FunctionReference.create', type_params=None, args=['-4280351851181442952', '-1550960115321255115', '-4578469861668566468', '59076

In [ ]:
metadsl_fn(1000)